THIS IS THE NOTEBEOOK WHERE WE ARE GOING TO FIRST INITIALIZE A MODEL AND THEN THEN EMBEDD THE DATA INTO THE RAG PIPELINE


In [ ]:
import pandas as pd
from langchain_core.documents import Document

# Load the FIFA matches data
df = pd.read_csv("data/Fifa_world_cup_matches.csv")  # adjust path if needed

# Quick peek
print(df.head())
print(df.columns)


converting each row of the data into text to embedded into the vector Store

In [ ]:
df.columns


In [ ]:
df.columns

In [ ]:
def row_to_text(row):
    # Adjust column names to your actual CSV headers
    return (
       
        f"FIFA World Cup 2022 Match {row['team1']} vs {row['team2']}. "
        f"Score: {row['number of goals team1']} - {row['number of goals team2']}. "
        f"Date: {row['date']}. "
        f"Category:{row['category']} ,"
        f"Possesion of {row['team1']} is {row['possession team1']} and {row['team2']} is {row['possession team2']} "
        f"{row['team1']} had {row['on target attempts team1']} On Target Attempts and {row['team2']} had  {row['on target attempts team2']} attempts"
         f"{row['team1']} had {row['assists team1']} assists and {row['team2']} had {row['assists team2']} assists"
    )

docs = []
for _, row in df.iterrows():
    text = row_to_text(row)
    metadata = {
        "team1": row["team1"],
        "team2": row["team2"],
        "category":row['category']
    }
    docs.append(Document(page_content=text, metadata=metadata))

len(docs), docs[0]


In [ ]:
print(docs)

INTIATING THE EMBEDDING MODEL

In [ ]:
from langchain_ollama import OllamaEmbeddings

embeddings = OllamaEmbeddings(
    model="llama3",
)



TEXT SPLITTER FOR THE RECAPS DATA


In [ ]:
from langchain_core.documents import Document
from langchain_community.vectorstores import FAISS
from langchain_ollama import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

def store_agent_responses_with_chunking(responses_dict, model_name="llama3"):
    """
    Store agent responses in a FAISS vector store using Ollama embeddings
    with recursive text splitting for better chunk management
    
    Args:
        responses_dict: Dictionary with response names as keys and response content as values
        model_name: Ollama model to use for embeddings (default: llama3)
    
    Returns:
        vector_store: FAISS vector store object
    """
    
    # Initialize Ollama embeddings
    embeddings = OllamaEmbeddings(model=model_name)
    
    # Initialize recursive character text splitter
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,           # Size of each chunk
        chunk_overlap=100,        # Overlap between chunks to maintain context
        separators=["\n\n", "\n", ". ", " ", ""]  # Split on these delimiters in order
    )
    
    # Create Document objects and chunk them
    docs = []
    for response_name, response_content in responses_dict.items():
        # Split the text into chunks
        chunks = text_splitter.split_text(response_content)
        
        # Create documents from chunks
        for i, chunk in enumerate(chunks):
            doc = Document(
                page_content=chunk,
                metadata={
                    "response_type": response_name,
                    "source": "agent_response",
                    "chunk_index": i,
                    "total_chunks": len(chunks)
                }
            )
            docs.append(doc)
    
    # Create FAISS vector store
    vector_store = FAISS.from_documents(docs, embedding=embeddings)
    
    print(f"✅ Successfully stored {len(docs)} chunks from {len(responses_dict)} responses!")
    return vector_store


# Usage example
responses = {
    "portugal_vs_morocco": result_3["messages"][-1].content,
    "possession_stats": result_2["messages"][-1].content,
    "uruguay_performance": resulted["messages"][-1].content,
}

# Create vector store with chunking
response_vector_store = store_agent_responses_with_chunking(responses)

# Create retriever
response_retriever = response_vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 5}
)

# Test searching through chunked responses
search_results = response_retriever.invoke("possession statistics")
for doc in search_results:
    print(f"Response: {doc.metadata['response_type']} (chunk {doc.metadata['chunk_index']})")
    print(f"Content: {doc.page_content[:200]}...")
    print("-" * 50)

In [ ]:
from langchain_community.vectorstores import FAISS
vector_store = FAISS.from_documents(docs, embedding=embeddings)


Now that the vector embeddings are completed we will now be doing a test for similarity search so that we can see what is actually going on here now using similiratiy_score_threshhold

similarity = “just give me closest stuff”

similarity_score_threshold = “closest stuff but drop trash”

mmr = “closest but also diverse (good for longer answers)”

In [ ]:
retriever = vector_store.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={
        "k": 10,                 # consider up to 10
        "score_threshold": 0.9   # filter out low similarity
    }
)



In [ ]:
retriever_s = vector_store.as_retriever(
    search_type="similarity",               # or "similarity_score_threshold", "mmr"
    search_kwargs={"k": 5}                  # top 5 matches
)


In [ ]:
prompt = "Tell me about the match between Portugal and Morocco"  # ⬅️ Add this

results = retriever_s.invoke(prompt)   # ⬅️ this replaces get_relevant_documents

for doc in results:
    print(doc.page_content)
    print(doc.metadata)
    print("-" * 40)

Now Setting Up The Actualy LLM and then seeing what can be done

In [ ]:
from langchain_ollama import ChatOllama
from langchain.agents import create_agent
from langchain.agents.middleware import SummarizationMiddleware
from langgraph.checkpoint.memory import InMemorySaver
from langchain_core.runnables import RunnableConfig
from langchain.tools import tool,ToolRuntime

In [ ]:
llm = ChatOllama(
    model="llama3.2:3b",  # or any local ollama model name you’ve pulled
    temperature=0.2,
)

checkpointer = InMemorySaver()

In [ ]:

from langchain_core.tools import create_retriever_tool

retriever_tool = create_retriever_tool(retriever_s,name='retriever_data',description='Usign this tool we will retrive the data from the vector store')



In [ ]:
@tool
def retrieve_data(query: str) -> str:
    """Retrieve World Cup match data from the vector store"""
    results = retriever_s.invoke(query)
    return "\n\n".join([doc.page_content for doc in results])

checkpointer = InMemorySaver()

agent = create_agent(
    model=llm,
    tools=[retrieve_data],
    checkpointer=checkpointer,
    system_prompt="""You are a World Cup football data assistant.
First call retrieve_data to retrieve context.
Use ONLY the data from the retrieve_data tool to answer the question.
If the data does not contain the answer, say so clearly."""
)


In [ ]:
config={"configurable": {"thread_id": "9"}}

result=agent.invoke({
    "messages":[{"role":"user","content":"Hey I am Ananda Prem How are you doing"}]
},config)

resulted=agent.invoke({
    "messages":[{"role":"user","content":"How Did Urugay Perform in this World Cup"}]
},config)

result_2=agent.invoke({
    "messages":[{"role":"user","content":"What would you say about the key moment where their fate in the worldcup was decided"}]
},config)



In [ ]:
print("Result 1:")
print(resulted["messages"][-1].content)

print("\nResult 2:")
print(result_2["messages"][-1].content)

print("\nResult 3:")
print(result["messages"][-1].content)

In [ ]:
print("Result 3 (Full):")
print(result_3["messages"][-1].content)


In [ ]:
result["messages"][-1].pretty_print()
result_2["messages"][-1].pretty_print()
result_3["messages"][-1].pretty_print()


In [ ]:
config={"configurable": {"thread_id": "1"}}

agent.invoke({"messages": "hi, my name is Nathan"}, config)
resp=agent.invoke({"messages": "Tell me about Portugals Match Against Morroco"}, config)
resp=agent.invoke({"messages": "What Would You tell About the possession did portugal deserve to win based off the possesion they maintained throught the match"}, config)
resp["messages"][-1].pretty_print()
resp=agent.invoke({"messages":"What is my name again ?"},config)
resp["messages"][-1].pretty_print()

Creating a Function to Get Input from a User instead Of hardCoding it